In [4]:
# Relevant module imports and installs
!pip install pulp pandas brotli
import pandas as pd
import pulp as plp
import sys 
import os
from collections import defaultdict


[notice] A new release of pip is available: 23.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [5]:
# Get the absolute path to the directory containing the Python file
module_path = os.path.abspath(os.path.join('..', '..'))

# Add the directory to sys.path
if module_path not in sys.path:
    sys.path.append(module_path)

# Import the data retrieval functions
from challenge_data import get_bootstrap_static_data, get_gameweek_data, update_with_gameweek_cost

# Get the actual GW2 data
bootstrap_data = get_bootstrap_static_data()
gw2_data = update_with_gameweek_cost(get_gameweek_data(2, bootstrap_data), '2024-25', 2)

In [6]:
# Get the number of players and their list of ids
player_ids = gw2_data['ID'].tolist()
player_count = len(player_ids)

# Set up the problem
model = plp.LpProblem("fpl-gw2-challenge-hindsight", plp.LpMaximize)

# Define the decision variables
lineup = [
    plp.LpVariable(f"lineup_{i}", lowBound=0, upBound=1, cat="Integer")
    for i in player_ids
]

# Define captain variables
captain = [
    plp.LpVariable(f"captain_{i}", lowBound=0, upBound=1, cat="Integer")
    for i in player_ids
]

# Set the objective function (the number of points scored by the team, with captain's points doubled)
model += plp.lpSum([lineup[i] * gw2_data.loc[i, 'Points'] for i in range(player_count)]) + \
         plp.lpSum([captain[i] * gw2_data.loc[i, 'Points'] for i in range(player_count)])

# Constraints

# Total number of players = 5 (1 goalkeeper + 4 forwards)
model += plp.lpSum(lineup) == 5

# Exactly one captain
model += plp.lpSum(captain) == 1

# Captain must be in the lineup
for i in range(player_count):
    model += captain[i] <= lineup[i]

# Exactly 1 Goalkeeper
model += plp.lpSum([lineup[i] for i in range(player_count) if gw2_data.loc[i, 'Position'] == 'Goalkeeper']) == 1

# Exactly 4 Forwards
model += plp.lpSum([lineup[i] for i in range(player_count) if gw2_data.loc[i, 'Position'] == 'Forward']) == 4

# Budget constraint: Total cost must be less than or equal to 35m
model += plp.lpSum([lineup[i] * gw2_data.loc[i, 'Cost'] for i in range(player_count)]) <= 35

# Solve the problem
model.solve(plp.PULP_CBC_CMD(msg=False))

# Function to print players by position
def print_players_by_position(players_dict):
    total_points = 0
    total_cost = 0
    for position in ['Goalkeeper', 'Forward']:
        if position in players_dict:
            print(f"\n{position}:")
            for player in players_dict[position]:
                captain_str = " (C)" if player['Captain'] else ""
                points = player['Points'] * (2 if player['Captain'] else 1)
                print(f"  {player['Name']}{captain_str} - {player['Team']} - Cost: {player['Cost']}m - Points: {points}")
                total_points += points
                total_cost += player['Cost']
    print(f"\nTotal Points: {round(total_points, 2)}")
    print(f"Total Cost: {round(total_cost, 2)}m")

# Print the results
print("Status:", plp.LpStatus[model.status])
selected_players = defaultdict(list)
for i in range(player_count):
    if lineup[i].value() == 1:
        player = gw2_data.loc[i]
        selected_players[player['Position']].append({
            'Name': player['Name'],
            'Team': player['Team'],
            'Cost': player['Cost'],
            'Points': player['Points'],
            'Captain': captain[i].value() == 1
        })

print("\nOptimal Lineup:")
print_players_by_position(selected_players)


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/91/3jnk9jzn3kx1mwsfzhddwbww0000gn/T/6b015b77bae240308f09eb2fa210c55d-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/91/3jnk9jzn3kx1mwsfzhddwbww0000gn/T/6b015b77bae240308f09eb2fa210c55d-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 637 COLUMNS
At line 7008 RHS
At line 7641 BOUNDS
At line 8896 ENDATA
Problem MODEL has 632 rows, 1254 columns and 3274 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 95 - 0.00 seconds
Cgl0004I processed model has 632 rows, 1254 columns (1254 integer (1254 of which binary)) and 3274 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 4 integers unsatisfied sum - 1.06667
Cbc00